<a href="https://colab.research.google.com/github/geelmens/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [4]:
import pandas as pd

## data.csv 파일 읽기
df = pd.read_csv('data.csv', encoding= 'utf_8')
print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


## 4-2 JSON 파일 읽기



In [9]:
import json
import pandas as pd

## data.json 파일 출력
with open('data.json', mode='r', encoding = 'utf-8') as f:
    data = json.load(f)

print(data)
## data.json 파일 DataFrame 읽기
df = pd.read_json('data.json', orient='records',encoding='utf_8',lines=False)
print(df)

{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [10]:
import re

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()
## 주민등록번호 패턴 생성
pattern = re.compile(r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******',content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode = 'w') as f:
    f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [11]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url,params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.030875205993652344, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-15T11:45', 'interval': 900, 'temperature_2m': 17.8}}
서울시 종로구의 현재 온도는 : 17.8°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [12]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0   180M      0 --:--:-- --:--:-- --:--:--  180M
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.9 MB/131 MB of archives.
After this operation, 448 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 /content/google-chrome-stable_current_amd64.deb google-chrome-stable amd64 141.0.7390.107-1 [121 MB]
Get:3 http://a

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
url = 'https://professor.knou.ac.kr/jaehwachung/index.do'
driver.get(url)

## 사이트 접속 대기
time.sleep(1)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text = tree.xpath('//title/text()')
print(title_text)

## 드라이버 종료
driver.quit()

['\n\t\tAlert \n\t\t\n\t']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [ ]:
import requests

## 데이터 수집 url 및 api key 설정


params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

## 데이터 수집

## 호출 성공/실패 출력



# 오픈소스기반 데이터 분석 과제

In [63]:
import requests
url = 'http://openapi.seoul.go.kr:8088/56587456746b79793331794548766d/json/energyUseDataSummaryInfo/1/5/'

def get_year_months():
  year_months = []
  for year in range(2015, 2024+1):
    for month in range(1,12+1):
      year_months.append(f'{year}{month:02d}')
  return year_months

responses = []

for year_month in get_year_months():
 url_full = url + year_month[0:4]+"/"+year_month[4:]
 response = requests.get(url_full)
 if response.status_code == 200:
  print("api 호출 성공")
  if year_month[4:] == "12":
    print(response.json())
 else:
  print(f"API 호출 실패: {response.status_code}")
 responses.append(response)

SyntaxError: cannot assign to subscript here. Maybe you meant '==' instead of '='? (ipython-input-3811084580.py, line 18)

In [62]:
df_list = []

for response in responses:
    data = response.json()['energyUseDataSummaryInfo']['row']
    df = pd.DataFrame(data)
    df_list.append(df)

df_all = pd.concat(df_list, ignore_index=True)
print(df_all)

    YEAR MON  MM_TYPE     CNT        EUS       EUS1       EUS2       ECO2_1  \
0   2015  01       개인  767791  193784708  194781915  204969429     -6090964   
1   2015  01       학교    1382  134955565  128707423  145561511     -2178902   
2   2015  01     종교단체      32     372270     363376     411363     -15099.5   
3   2015  01     소상공인    2058   10249618    9927610   11039593    -233983.5   
4   2015  01  공동주택관리소    1747  327102096  329852334  351694752    -13671447   
5   2015  02       개인  774620  189974230  193611430  200055533   -6859251.5   
6   2015  02       기업   16966  523325778  542435234  567181200    -31482439   
7   2015  02     소상공인    2153    9763412    9786829   10214569      -237287   
8   2015  02     공공기관    2981  230038970  227707037  239494601     -3561849   
9   2015  02     종교단체      32     336476     341646     366928       -17811   
10  2015  03       학교    3903  145577747  140333207  148915260       673170   
11  2015  03     공공기관    2048  208343249  206515245 

In [60]:
import pandas as pd

data = responses.json()['energyUseDataSummaryInfo']['row']
df = pd.DataFrame(data)
print(df)

AttributeError: 'list' object has no attribute 'json'

In [33]:
def get_year_months():
  year_months = []
  for year in range(2015, 2024+1):
    for month in range(1,12+1):
      year_months.append(f'{year}{month:02d}')
  return year_months

x = get_year_months()
print(x)

['201501', '201502', '201503', '201504', '201505', '201506', '201507', '201508', '201509', '201510', '201511', '201512', '201601', '201602', '201603', '201604', '201605', '201606', '201607', '201608', '201609', '201610', '201611', '201612', '201701', '201702', '201703', '201704', '201705', '201706', '201707', '201708', '201709', '201710', '201711', '201712', '201801', '201802', '201803', '201804', '201805', '201806', '201807', '201808', '201809', '201810', '201811', '201812', '201901', '201902', '201903', '201904', '201905', '201906', '201907', '201908', '201909', '201910', '201911', '201912', '202001', '202002', '202003', '202004', '202005', '202006', '202007', '202008', '202009', '202010', '202011', '202012', '202101', '202102', '202103', '202104', '202105', '202106', '202107', '202108', '202109', '202110', '202111', '202112', '202201', '202202', '202203', '202204', '202205', '202206', '202207', '202208', '202209', '202210', '202211', '202212', '202301', '202302', '202303', '202304',

In [48]:
YYYYMM = '201502'
date = YYYYMM[0:4]+"/"+YYYYMM[4:]


url_pre = 'http://openapi.seoul.go.kr:8088/56587456746b79793331794548766d/json/energyUseDataSummaryInfo/1/5/'
url_full = url_pre + date
print(url_full)

http://openapi.seoul.go.kr:8088/56587456746b79793331794548766d/json/energyUseDataSummaryInfo/1/5/2015/02
